In [46]:
import pandas as pd
import numpy as np
import flirt
import os
import jupyter
import ipywidgets

# 1. Load WESAD and nurses dataset

In [47]:
# load data
df = pd.read_parquet('data-input/dataset_nurse_acc_bvp_eda_hr_temp.parquet')
df

,x,y,z,BVP,EDA,HR,TEMP,label,subject
2020-07-08 08:30:00.000000,-52.0,15.0,-35.0,18.47,5.773421,99.05,31.63,1,15
2020-07-08 08:30:00.015625,NaN,NaN,NaN,10.63,NaN,NaN,NaN,1,15
2020-07-08 08:30:00.031250,-52.0,15.0,-35.0,-0.95,NaN,NaN,NaN,1,15
2020-07-08 08:30:00.046875,NaN,NaN,NaN,-16.12,NaN,NaN,NaN,1,15
2020-07-08 08:30:00.062500,-52.0,15.0,-35.0,-33.68,NaN,NaN,NaN,1,15
...,...,...,...,...,...,...,...,...,...
2020-07-23 12:25:57.250000,NaN,NaN,NaN,NaN,0.129352,NaN,NaN,1,F5
2020-07-23 12:25:57.500000,NaN,NaN,NaN,NaN,0.125510,NaN,NaN,1,F5
2020-07-23 12:25:57.750000,NaN,NaN,NaN,NaN,0.119106,NaN,NaN,1,F5
2020-07-23 12:25:58.000000,NaN,NaN,NaN,NaN,0.116545,NaN,NaN,1,F5


In [48]:
# split dfs when we encounter a timestamp break
df['groups'] = (df.index.to_series().diff().dt.seconds > 1).cumsum()
df

,x,y,z,BVP,EDA,HR,TEMP,label,subject,groups
2020-07-08 08:30:00.000000,-52.0,15.0,-35.0,18.47,5.773421,99.05,31.63,1,15,0
2020-07-08 08:30:00.015625,NaN,NaN,NaN,10.63,NaN,NaN,NaN,1,15,0
2020-07-08 08:30:00.031250,-52.0,15.0,-35.0,-0.95,NaN,NaN,NaN,1,15,0
2020-07-08 08:30:00.046875,NaN,NaN,NaN,-16.12,NaN,NaN,NaN,1,15,0
2020-07-08 08:30:00.062500,-52.0,15.0,-35.0,-33.68,NaN,NaN,NaN,1,15,0
...,...,...,...,...,...,...,...,...,...,...
2020-07-23 12:25:57.250000,NaN,NaN,NaN,NaN,0.129352,NaN,NaN,1,F5,161
2020-07-23 12:25:57.500000,NaN,NaN,NaN,NaN,0.125510,NaN,NaN,1,F5,161
2020-07-23 12:25:57.750000,NaN,NaN,NaN,NaN,0.119106,NaN,NaN,1,F5,161
2020-07-23 12:25:58.000000,NaN,NaN,NaN,NaN,0.116545,NaN,NaN,1,F5,161


In [49]:
# data cleaning

clean_dfs = []

df_chunk = np.nan

# loop over groups
for i in df['groups'].unique().tolist():
    df_chunk = df[df['groups'] == i]
    
    # remove group if no valid HR, EDA, or TEMP
    c1 = df_chunk['HR'].isnull().all()
    c2 = df_chunk['EDA'].isnull().all()
    c3 = df_chunk['TEMP'].isnull().all()

    if c1 or c2 or c3:
        continue

    # remove rows with where BVP is NaN - 64hz
    df_chunk = df_chunk[df_chunk['BVP'].notna()]

    # remove rows where there are 2 or more consecutive x missing - 32hz
    k = 2
    i = df_chunk.x.isnull()
    m = ~(df_chunk.groupby(i.ne(i.shift()).cumsum().values).x.transform('size').ge(k) & i)
    df_chunk = df_chunk[m]

    # remove rows where there are 16 or more  consecutive EDA missing - 4hz
    k = 16
    i = df_chunk.EDA.isnull()
    m = ~(df_chunk.groupby(i.ne(i.shift()).cumsum().values).EDA.transform('size').ge(k) & i)
    df_chunk = df_chunk[m]

    # remove rows where there are 16 or more consecutive TEMP missing - 4hz
    k = 16
    i = df_chunk.TEMP.isnull()
    m = ~(df_chunk.groupby(i.ne(i.shift()).cumsum().values).TEMP.transform('size').ge(k) & i)
    df_chunk = df_chunk[m]

    # remove rows where there are 64 or more consecutive HR missing - 1hz
    k = 64
    i = df_chunk.HR.isnull()
    m = ~(df_chunk.groupby(i.ne(i.shift()).cumsum().values).HR.transform('size').ge(k) & i)
    df_chunk = df_chunk[m]
    
    clean_dfs.append(df_chunk)

df = pd.concat(clean_dfs)

In [50]:
df

,x,y,z,BVP,EDA,HR,TEMP,label,subject,groups
2020-07-08 08:30:00.000000,-52.0,15.0,-35.0,18.47,5.773421,99.05,31.63,1,15,0
2020-07-08 08:30:00.015625,NaN,NaN,NaN,10.63,NaN,NaN,NaN,1,15,0
2020-07-08 08:30:00.031250,-52.0,15.0,-35.0,-0.95,NaN,NaN,NaN,1,15,0
2020-07-08 08:30:00.046875,NaN,NaN,NaN,-16.12,NaN,NaN,NaN,1,15,0
2020-07-08 08:30:00.062500,-52.0,15.0,-35.0,-33.68,NaN,NaN,NaN,1,15,0
...,...,...,...,...,...,...,...,...,...,...
2020-07-23 12:25:36.687500,-45.0,-38.0,27.0,28.30,NaN,NaN,NaN,1,F5,161
2020-07-23 12:25:36.703125,NaN,NaN,NaN,19.59,NaN,NaN,NaN,1,F5,161
2020-07-23 12:25:36.718750,-51.0,-52.0,23.0,9.46,NaN,NaN,NaN,1,F5,161
2020-07-23 12:25:36.734375,NaN,NaN,NaN,-0.96,NaN,NaN,NaN,1,F5,161


# 2. Function for getting features from FLIRT

In [51]:
def get_features_inner(df, columns_list, prefix, window_length, window_step_size, frequency):
    df = df[columns_list]
    df = df.dropna()

    features = flirt.get_acc_features(df,
                                      window_length=window_length, 
                                      window_step_size=window_step_size,
                                      data_frequency=frequency)
    features = features.add_prefix(prefix)
    return features

In [52]:
def get_features(subject, label, data, window_length, window_step_size):

    # we need to set a correct datetime index with 64hz, which is 15625000 nanoseconds,
    # otherwise Flirt will create a wrong timeindex
    #time_index = pd.date_range(start=0, periods=len(data), freq='15625000N')
    #data = data.set_index(time_index)
    
    # calculate features
    bvp_features = get_features_inner(data, ['BVP'], 'bvp_', window_length, window_step_size, 64)
    acc_features = get_features_inner(data, ['x', 'y', 'z'], 'acc_', window_length, window_step_size, 32)
    eda_features = get_features_inner(data, ['EDA'], 'eda_', window_length, window_step_size, 4)
    temp_features = get_features_inner(data, ['TEMP'], 'temp_', window_length, window_step_size, 4)
    hr_features = get_features_inner(data, ['HR'], 'hr_', window_length, window_step_size, 1)

    # merge
    res = pd.concat([bvp_features, acc_features, eda_features, temp_features, hr_features], axis=1)
    
    # add subject and label column
    res['subject'] = subject
    res['label'] = label

    return res

# 3. Calculate features for the whole dataset

In [53]:
df.shape

(17074048, 10)

In [54]:
df.describe()

,x,y,z,BVP,EDA,HR,TEMP,label,groups
count,8.537105e+06,8.537105e+06,8.537105e+06,1.707405e+07,1.067279e+06,266924.000000,1.067279e+06,1.707405e+07,1.707405e+07
mean,-2.787857e+01,-8.684977e-01,2.380013e+01,-1.270045e-03,2.524590e+00,84.340621,3.213599e+01,8.393081e-01,8.118395e+01
std,3.223118e+01,3.215962e+01,2.938650e+01,1.015688e+02,4.792756e+00,13.904804,2.225368e+00,3.672465e-01,4.466536e+01
min,-1.280000e+02,-1.280000e+02,-1.280000e+02,-3.096160e+03,0.000000e+00,49.000000,2.347000e+01,0.000000e+00,0.000000e+00
25%,-5.200000e+01,-1.700000e+01,4.000000e+00,-2.071000e+01,1.447730e-01,75.870000,2.999000e+01,1.000000e+00,3.500000e+01
50%,-3.300000e+01,1.000000e+00,2.600000e+01,8.800000e-01,4.576470e-01,82.150000,3.263000e+01,1.000000e+00,8.200000e+01
75%,-1.400000e+01,1.500000e+01,5.000000e+01,2.184000e+01,2.378491e+00,91.030000,3.407000e+01,1.000000e+00,1.180000e+02
max,1.270000e+02,1.270000e+02,1.270000e+02,2.506460e+03,4.794970e+01,180.230000,3.705000e+01,1.000000e+00,1.610000e+02


In [56]:
%%time

result_dfs = []

window_length = 60
window_step_size = 10

# loop over groups
for i in df['groups'].unique().tolist():
    df_chunk = df[df['groups'] == i]
    
    subject = df_chunk['subject'][0]
    label = df_chunk['label'][0]
    
    df_chunk = df_chunk.drop(columns=['groups', 'subject', 'label'])
    
    res_df_chunk = get_features(subject, label, df_chunk, window_length, window_step_size)
    result_dfs.append(res_df_chunk)

res = pd.concat(result_dfs)

ACC features:   0%|          | 0/23 [00:00<?, ?it/s]

Unable to remove memmapped file


ACC features:   0%|          | 0/23 [00:00<?, ?it/s]

Unable to remove memmapped file


ACC features:   0%|          | 0/23 [00:00<?, ?it/s]

...

Unable to remove memmapped file


ACC features:   0%|          | 0/22 [00:00<?, ?it/s]

Unable to remove memmapped file


ACC features:   0%|          | 0/22 [00:00<?, ?it/s]

Unable to remove memmapped file
CPU times: total: 4min 40s
Wall time: 31min 9s


In [57]:
res

,bvp_BVP_mean,bvp_BVP_std,bvp_BVP_min,bvp_BVP_max,bvp_BVP_ptp,bvp_BVP_sum,bvp_BVP_energy,bvp_BVP_skewness,bvp_BVP_kurtosis,bvp_BVP_peaks,...,hr_l2_n_sign_changes,hr_l2_iqr,hr_l2_iqr_5_95,hr_l2_pct_5,hr_l2_pct_95,hr_l2_entropy,hr_l2_perm_entropy,hr_l2_svd_entropy,subject,label
datetime,,,,,,,,,,,,,,,,,,,,,
2020-07-08 08:31:00,0.376078,207.186980,-1336.28,1333.44,2669.72,1444.14,1.648381e+08,-0.210934,8.208982,174,...,0.0,16.1900,20.1850,78.669,98.8540,4.090539,0.537376,0.040885,15,1
2020-07-08 08:31:10,0.813401,205.085789,-1336.28,1470.89,2807.17,3123.46,1.615136e+08,0.453604,10.256058,172,...,0.0,11.9525,18.4160,78.669,97.0850,4.091445,0.879136,0.054576,15,1
2020-07-08 08:31:20,-1.458391,183.187930,-823.18,1470.89,2294.07,-5600.22,1.288702e+08,1.219684,12.753530,178,...,0.0,6.7350,14.3385,78.669,93.0075,4.092899,0.998001,0.068318,15,1
2020-07-08 08:31:30,-3.841466,157.598661,-823.18,1470.89,2294.07,-14751.23,9.543204e+07,0.723881,14.803648,188,...,0.0,6.7350,13.9260,78.669,92.5950,4.092931,0.934849,0.067763,15,1
2020-07-08 08:31:40,-1.048693,194.983749,-1061.23,1470.89,2532.12,-4026.98,1.459959e+08,0.870929,11.565500,181,...,0.0,11.0850,21.9930,78.669,100.6620,4.090714,0.669996,0.055197,15,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-07-23 12:25:50,-0.073785,72.309958,-311.70,362.36,674.06,-220.84,1.564961e+07,-0.293683,3.663040,175,...,0.0,5.6750,7.4300,115.000,122.4300,3.849875,0.439497,0.014628,F5,1
2020-07-23 12:26:00,-0.189541,77.083182,-311.70,362.36,674.06,-445.99,1.398118e+07,-0.180828,3.137394,136,...,0.0,4.0000,5.9700,116.474,122.4440,3.610759,0.522559,0.015543,F5,1
2020-07-23 12:26:10,-0.291518,82.407074,-311.70,362.36,674.06,-499.37,1.163300e+07,-0.139853,2.919981,103,...,0.0,2.0750,3.5680,118.911,122.4790,3.295781,0.650022,0.015997,F5,1


In [58]:
# handle NANs
res = res.dropna()

In [59]:
res

,bvp_BVP_mean,bvp_BVP_std,bvp_BVP_min,bvp_BVP_max,bvp_BVP_ptp,bvp_BVP_sum,bvp_BVP_energy,bvp_BVP_skewness,bvp_BVP_kurtosis,bvp_BVP_peaks,...,hr_l2_n_sign_changes,hr_l2_iqr,hr_l2_iqr_5_95,hr_l2_pct_5,hr_l2_pct_95,hr_l2_entropy,hr_l2_perm_entropy,hr_l2_svd_entropy,subject,label
datetime,,,,,,,,,,,,,,,,,,,,,
2020-07-08 08:31:00,0.376078,207.186980,-1336.28,1333.44,2669.72,1444.14,1.648381e+08,-0.210934,8.208982,174,...,0.0,16.1900,20.1850,78.669,98.8540,4.090539,0.537376,0.040885,15,1
2020-07-08 08:31:10,0.813401,205.085789,-1336.28,1470.89,2807.17,3123.46,1.615136e+08,0.453604,10.256058,172,...,0.0,11.9525,18.4160,78.669,97.0850,4.091445,0.879136,0.054576,15,1
2020-07-08 08:31:20,-1.458391,183.187930,-823.18,1470.89,2294.07,-5600.22,1.288702e+08,1.219684,12.753530,178,...,0.0,6.7350,14.3385,78.669,93.0075,4.092899,0.998001,0.068318,15,1
2020-07-08 08:31:30,-3.841466,157.598661,-823.18,1470.89,2294.07,-14751.23,9.543204e+07,0.723881,14.803648,188,...,0.0,6.7350,13.9260,78.669,92.5950,4.092931,0.934849,0.067763,15,1
2020-07-08 08:31:40,-1.048693,194.983749,-1061.23,1470.89,2532.12,-4026.98,1.459959e+08,0.870929,11.565500,181,...,0.0,11.0850,21.9930,78.669,100.6620,4.090714,0.669996,0.055197,15,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-07-23 12:25:50,-0.073785,72.309958,-311.70,362.36,674.06,-220.84,1.564961e+07,-0.293683,3.663040,175,...,0.0,5.6750,7.4300,115.000,122.4300,3.849875,0.439497,0.014628,F5,1
2020-07-23 12:26:00,-0.189541,77.083182,-311.70,362.36,674.06,-445.99,1.398118e+07,-0.180828,3.137394,136,...,0.0,4.0000,5.9700,116.474,122.4440,3.610759,0.522559,0.015543,F5,1
2020-07-23 12:26:10,-0.291518,82.407074,-311.70,362.36,674.06,-499.37,1.163300e+07,-0.139853,2.919981,103,...,0.0,2.0750,3.5680,118.911,122.4790,3.295781,0.650022,0.015997,F5,1


In [60]:
# store as parquet

if not os.path.isdir('data-input'):
    os.makedirs('data-input')

res.to_parquet('data-input/flirt-nurse-acc-bvp-eda-hr-temp-'+str(window_length)+'-'+str(window_step_size)+'.parquet')